# Ehsan Kouchaki
**In this homework, you will learn how to:**
- Implement a 2-class classification neural network 
- Use units with a non-linear activation function, such as tanh 
- Compute the cross entropy loss 
- Implement forward and backward propagation
- Update weights using gradient descent
- ...

We will start by loading the packages:

In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline

Now, we load the dataset:

In [2]:
# load dataset

filename = 'cardio_dataset.csv'
    
reader = csv.reader(open("cardio_dataset.csv", "r"), delimiter=";")
x = list(reader)

dataset_1 = np.array(x).astype("float")

From now implement your model.
<br>
Please add cells and explain yours developing steps and your results.

<div dir='rtl'style="text-align: right;font-family:verdana;">
برای توضیحات از زبان فارسی استفاده نمایید.
<br>
موفق باشید

</div>

<div dir="rtl">
در این بخش نخست داده ها به دو بخش داده های آموزشی و داده های تست تقسیم شده است. به این منظور 60000 داده نخست به عنوان داده های آموزشی و 10000 داده آخر به عنوان داده های تست در نظر گرفته شدند. پس از آن داده ها اصلاح شده است. به این صورت که نخست داده ها ترانهاده شده و پس از آن سطر اول داده ها که مربوط به شماره ردیف آنها است حذف شده است. به این ترتیب همانند مواردی که در کلاس درس گفته شد ماتریس داده هابه این صورت تشکیل شد که در هر ستون ویژگی های یک داده قرار داده شده است. به این ترتیب تعداد ردیف های ماتریس داده ها برابر با تعداد ویژگی ها و تعداد ستون های آن برابر با تعداد نمونه های آموزشی شده است همچنین ردیف آخر که مربوط به برچسب داده ها است با نام جداگانه به عنوان بردار برچسب جدا سازی و نامگذاری شده است

In [3]:
trains_number = 60000
dataset_train = dataset_1[0:trains_number].T
dataset_test = dataset_1[trains_number:70000].T

X_train = dataset_train[1:12]    # train data
X_test = dataset_test[1:12]      # test data

Y_train = dataset_train[12]        # train ground truth
Y_test = dataset_test[12]          # test ground truth

print("The shape of train data is: ", X_train.shape)
print("The shape of test data is: ", X_test.shape)
print("The shape of train ground truth is: ", Y_train.shape)
print("The shape of test ground truth is: ", Y_test.shape)

The shape of train data is:  (11, 60000)
The shape of test data is:  (11, 10000)
The shape of train ground truth is:  (60000,)
The shape of test ground truth is:  (10000,)


## Data normalization
<div dir="rtl">
با توجه به اینکه ویژگیهای داده ها پراکندگی زیادی دارند، در این بخش داده ها با استفاده از واریانس میانگین که در کلاس درس گفته شد نرمال سازی شده اند

In [4]:
for i in range(X_train.shape[0]):
    mean_train = np.mean(X_train[i])
    X_train[i] -= mean_train
    mean_test = np.mean(X_test[i])
    X_test[i] -= mean_test

for i in range(X_train.shape[0]):
    var_train = np.var(X_train[i])
    X_train[i] /= var_train
    var_test = np.var(X_test[i])
    X_test[i] /= var_test

## Activation functions
<div dir="rtl">
در این بخش توابع فعال ساز رلو و سیگمویید برای استفاده تعریف شده اند

In [5]:
def sigmoid(z):
    sigma = 1/(1 + np.exp(-z))
    return sigma

def dsigmoid(z):
    sigma = 1/(1 + np.exp(-z))
    sigmap = sigma * (1 - sigma)
    return sigmap

def relu(z):
    rel = np.maximum(0, z)
    return rel

def drelu(z):
    z[z<=0] = 0
    z[z>0] = 1
    return z
             

## Affine and activation modules
<div dir="rtl">
در این بخش تابعهایی برای ماژولهای مورد استفاده برای ساختن شبکه عصبی نوشته شده است

In [6]:
def affine_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

def Relu_forward(Z):
    A = relu(Z)
    cache = Z
    return A, cache

def Sigmoid_forward(Z):
    A = sigmoid(Z)
    cache = Z
    return A, cache

def compute_cost(Y_hat, Y):
    cost = - np.sum(np.multiply(np.log(Y_hat),Y) + np.multiply(np.log(1 - Y_hat),(1 - Y))) / Y_hat.shape[1]
    return cost

def affine_backward(dZ, cache):
    A_l_1, W_l, b_l = cache[0], cache[1], cache[2]
    m = A_l_1.shape[1]
    
    dW_l = 1 / m * np.dot(dZ, A_l_1.T)
    db_l = 1 / m * np.sum(dZ, axis = 1, keepdims = True)
    dA_l_1 = np.dot(W_l.T, dZ)
    return dA_l_1, dW_l, db_l

def Relu_backward(dA, cache):
    Z_l = cache
    dZ = dA * drelu(Z_l)
    return dZ

def Sigmoid_backward(dA, cache):
    Z_l = cache
    dZ = dA * dsigmoid(Z_l)
    return dZ
    


## Network design 
<div dir="rtl">
در این بخش شبکه عصبی با مشخص کردن تعداد لایه ها و تعداد نرون ها در هر لایه مخفی طراحی شده است. برنامه این قابلیت را دارد که در همین قسمت به تعداد دلخواه لایه ها و تعداد نرون ها انتخاب و تعیین شود 

In [7]:
Layers = [10, 5, 3]    # تعداد لایه ها مخفی به همراه تعدادنرون ها در هر لایه 
Layers.insert(0,X_train.shape[0])
Layers.append(1)
print("همه لایه ها و تعداد نرون ها شامل لایه ورودی و خروجی:\n\n ", Layers)

همه لایه ها و تعداد نرون ها شامل لایه ورودی و خروجی:

  [11, 10, 5, 3, 1]


## Train 
<div dir="rtl">
در این بخش آموزش شبکه عصبی نوشته شده است
بر اساس توضیحات ارایه شده در کلاس این بخش به صورت ماژولار و با استفاده از بلوکهای جداگانه برای محاسبات به جلو و پس انتشار خطا و ذخیره خروجی های مورد نیاز به صورت مستقیم و در قالب کش نوشته شده است

In [8]:
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

# مقدار دهی اولیه وزنها و بایاس ها
Weights = []                  # لیست نگهدارنده وزنها
biases = []                   # لیست نگهدارنده بایاسها

for l in range(1, len(Layers)):
    # روش تصادفی
    Weights.append(np.random.randn(Layers[l], Layers[l-1]) * 0.01)
    # روش He
    #Weights.append(np.random.normal(loc=0, scale=np.sqrt(2/Layers[l-1]), size=(Layers[l], Layers[l-1])))
    
    biases.append(np.zeros((Layers[l], 1)))
    
L = len(Weights)     # the number of layers

losses = []
Epochs = 30000
for epoch in range(Epochs):
    
    learning_rate = 13 / (epoch + 1)
    
    A = X_train

    caches = []
    # انتشار به جلو برای لایه های مخفی
    for l in range(L-1):
        Z, cache = affine_forward(A, Weights[l], biases[l])
        caches.append(cache)
        A, cache = Relu_forward(Z)
        caches.append(cache)
        
    # انتشار به جلو برای لایه آخر
    Z, cache = affine_forward(A, Weights[L-1], biases[L-1])
    caches.append(cache)
    Y_hat , cache = Sigmoid_forward(Z)
    caches.append(cache)
    
    # محاسبه هزینه
    #Y_train = Y_train.reshape(Y_hat.shape)    # با این کار دو بردار هم اندازه می شود
    Y_hat += 0.0000001*(Y_hat == 0) - 0.0000001*(Y_hat == 1)    #  اطمینلن از اینکه در تابع هزینه از صفر لگاریتم گرفته نشود
    cost = compute_cost(Y_hat, Y_train)

    grads = {}

    # پس انتشار برای لایه آخر
    current_cache = caches[2 * L - 1]   # Z of the last layer
    dZ = Y_hat-Y_train
    #dZ = Sigmoid_backward(dA, current_cache)
    current_cache = caches[2 * L - 2]
    dA, dW, db = affine_backward(dZ, current_cache)
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = dA, dW, db

    # پس انتشار برای لایه های مخفی
    for l in range(L-2,-1,-1):
        current_cache = caches[2 * l + 1]   # Z of the l'th layer
        dZ =Relu_backward(dA, current_cache)
        current_cache = caches[2 * l]
        dA, dW, db = affine_backward(dZ, current_cache)
        grads["dA" + str(l)], grads["dW" + str(l+1)], grads["db" + str(l+1)] = dA, dW, db

    # بروز کردن وزنها و بایاسها
    for l in range(L):
        Weights[l] = Weights[l] - learning_rate * grads["dW" + str(l+1)]
        biases[l] = biases[l] - learning_rate * grads["db" + str(l+1)]

    # Print the cost
    if epoch % int(Epochs / 20) == 0:
        print ("Cost after iteration %i: %f" %(epoch, cost))
    
    losses.append(cost)
    
plt.plot(losses)

predictions = Y_hat > 0.5   # خروجی های بزرگتر از نیم به عنوان 1 در نظر گرفته می شوند
print("predictions mean = " + str(np.mean(predictions)))

# Accuracy
print ('Accuracy: %d' % float((np.dot(Y_train,predictions.T)
                               + np.dot(1-Y_train,1-predictions.T))/float(Y_train.size)*100) + '%')
print(learning_rate)

Cost after iteration 0: 0.693147


KeyboardInterrupt: 

## Test

In [14]:
A = X_test
for l in range(L-1):
    Z, cache = affine_forward(A, Weights[l], biases[l])
    A, cache = Relu_forward(Z)

Z, cache = affine_forward(A, Weights[L-1], biases[L-1])
Y_hat , cache = Sigmoid_forward(Z)

predictions = Y_hat > 0.5
print("predictions mean = " + str(np.mean(predictions)))

# Accuracy
print ('Accuracy: %d' % float((np.dot(Y_test,predictions.T) + np.dot(1-Y_test,1-predictions.T))/float(Y_test.size)*100) + '%')

predictions mean = 0.4944
Accuracy: 71%


In [15]:
import pickle

with open("Weights0.txt", "wb") as fp:   #Pickling
    pickle.dump(Weights, fp)

with open("Weights0.txt", "rb") as fp:   # Unpickling
    w = pickle.load(fp)

with open("biases0.txt", "wb") as fp:   #Pickling
    pickle.dump(biases, fp)

with open("biases0.txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)

with open("losses0.txt", "wb") as fp:   #Pickling
    pickle.dump(losses, fp)

with open("losses0.txt", "rb") as fp:   # Unpickling
    l = pickle.load(fp)


<div dir="rtl">
    
# جمع بندی و نتایج
    در این تکلیف با استفاده از شبکه عصبی مساله دسته بندی افراد برای تشخیص بیماری قلبی با استفاده از داده های کاردیوگرافی حل شد.
 ##   ویژگی های مدل حاضر
#### ۱- این مدل با استفاده از ماژولهای بیشرو و پس رو و ذخیره موارد مورد نیاز در انتشار به جلو به عنوان کش ساخته شد
#### ۲-  این برنامه قابلیت تعیین شبکه با تعداد لایه های دلخواه و تعداد نرون ها را دارد بدون اینکه نیاز به نوشتن برنامه جدیدی برای هر حالت باشد
#### ۳- در این برنامه وزدن دهی اولیه به روش هی نیز انجام شد.
#### ۴- در این مدل از نرخ یادگیری متغیر که نسبت عکس با شماره ایپاک دارد استفاده شد تا علاوه بر افزایش سرعت همگرایی در گامهای اولیه در گامهای نهایی با کاهش نرخ یادگیری دقت همگرایی افزایش یابد
#### 
    
#  نتایج
## تاثیر تعداد لایه ها
#### در این تکلیف مساله با تعیین تعداد لایه های مختلف شبکه عصبی حل شد و هر بار نتایج مختلفی به دست آمد. با در نظر گرفتن شبکه به صورت ساده دارای یک یا دو لایه مخفی نتایج خوبی از نظر دقت به دست نیامد به عنوان نمونه در نظر نگرفتن لایه مخفی برای مساله شبکه را تبدیل به یک سیستم مانند لاجیستیک رگرسیون می کند و عملکرد ضعیفی خواهد داشت. از طرفی تعداد لایه های مخفی از 3 لایه بیشتر نه تنها باعث سنگینتر شدن روند حل و افزایش زمان حل شد بلکه تاثیر مثبتی در دقت شبکه ایجاد نکرد. بهترین نتیجه با در نظر گرفتن سه لایه مخفی برای این مساله به دست آمد
## تاثیر تعداد نرونها
#### برای حل این مساله تعداد مختلفی نرون برای لایه های مختلف شبکه در نظر گرفته شد با انتخاب تعداد نرون کم در لایه ها مثلا در حد ۳ یا چهار نرون شبکه از قدرت خوبی برخوردار نبود از طرفی تعداد نرون بیش از حد زیاد نیز تاثیری در عملکرد مثبت شبکه ایجاد  نکرد. درنهایت با انتخاب تعداد نرون ها به صورت مخروطی و با اعداد موجود در همین خروجی برنامه حاضر تقریبا بهترین پاسخها به دست آمد
## تاثیر نوع مقدار دهی وزنها
#### در این برنامه از دو روش تصادفی و روش هی برای مقدار دهی اولیه وزنها استفاده شد. در نهایت برای این مساله استفاده از روش   هی موجب ایجاد مقادیر وزنهای بزرگتری در ایپاکهای اولیه و همچنین مقدار هزینه بزرگتر شد ولی به تدریج مقادیر هزینه کاهش یافت. در نهایت استفاده از دو روش مقدار دهی تصادفی و روش هی تفاوت چندانی در دقت نهایی داده های آموزش و تست برای این مساله ایجاد نکرد
## تاثیر تعداد داده های آموزشی
#### در این برنامه مساله با تقسیم داده ها به آموزشی و تست برای چند حالت مختلف انجام شد در صورت انتخاب تعداد داده های آموزشی به تعداد کم شبکه به طور مطلوبی آموزش نمیدید از طرفی انتخاب بیش از حد معمول تعداد داده های آموزشی موجب بر هم خوردن تعادل نسبت بین داده های آموزشی و تست می گردد و موجب می شود معیار درستی برای دقت عملکرد شبکه به دست نیاید در این مساله داده های آموزشی به تعداد 60000 در نهایت نتایج مطلوبی به دست داد هرجند انتخاب داده ها به میزان کمتر و در حدود بین 40000 تا 60000 تفاوت چندانی در نتایج ایجاد نکرد    

    
# نتایج بهترین وزنها و بایاسهای به دست آمده ذخیره شده و به پیوست ارسال شده است 
    
    
    
    <div dir="rtl">
    
# جمع بندی و نتایج
    در این تکلیف با استفاده از شبکه عصبی مساله دسته بندی افراد برای تشخیص بیماری قلبی با استفاده از داده های کاردیوگرافی حل شد.
 ##   ویژگی های مدل حاضر
#### ۱- این مدل با استفاده از ماژولهای بیشرو و پس رو و ذخیره موارد مورد نیاز در انتشار به جلو به عنوان کش ساخته شد
#### ۲-  این برنامه قابلیت تعیین شبکه با تعداد لایه های دلخواه و تعداد نرون ها را دارد بدون اینکه نیاز به نوشتن برنامه جدیدی برای هر حالت باشد
#### ۳- در این برنامه وزدن دهی اولیه به روش هی نیز انجام شد.
#### ۴- در این مدل از نرخ یادگیری متغیر که نسبت عکس با شماره ایپاک دارد استفاده شد تا علاوه بر افزایش سرعت همگرایی در گامهای اولیه در گامهای نهایی با کاهش نرخ یادگیری دقت همگرایی افزایش یابد
#### 
    
#  نتایج
## تاثیر تعداد لایه ها
#### در این تکلیف مساله با تعیین تعداد لایه های مختلف شبکه عصبی حل شد و هر بار نتایج مختلفی به دست آمد. با در نظر گرفتن شبکه به صورت ساده دارای یک یا دو لایه مخفی نتایج خوبی از نظر دقت به دست نیامد به عنوان نمونه در نظر نگرفتن لایه مخفی برای مساله شبکه را تبدیل به یک سیستم مانند لاجیستیک رگرسیون می کند و عملکرد ضعیفی خواهد داشت. از طرفی تعداد لایه های مخفی از 3 لایه بیشتر نه تنها باعث سنگینتر شدن روند حل و افزایش زمان حل شد بلکه تاثیر مثبتی در دقت شبکه ایجاد نکرد. بهترین نتیجه با در نظر گرفتن سه لایه مخفی برای این مساله به دست آمد
## تاثیر تعداد نرونها
#### برای حل این مساله تعداد مختلفی نرون برای لایه های مختلف شبکه در نظر گرفته شد با انتخاب تعداد نرون کم در لایه ها مثلا در حد ۳ یا چهار نرون شبکه از قدرت خوبی برخوردار نبود از طرفی تعداد نرون بیش از حد زیاد نیز تاثیری در عملکرد مثبت شبکه ایجاد  نکرد. درنهایت با انتخاب تعداد نرون ها به صورت مخروطی و با اعداد موجود در همین خروجی برنامه حاضر تقریبا بهترین پاسخها به دست آمد
## تاثیر نوع مقدار دهی وزنها
#### در این برنامه از دو روش تصادفی و روش هی برای مقدار دهی اولیه وزنها استفاده شد. در نهایت برای این مساله استفاده از روش   هی موجب ایجاد مقادیر وزنهای بزرگتری در ایپاکهای اولیه و همچنین مقدار هزینه بزرگتر شد ولی به تدریج مقادیر هزینه کاهش یافت. در نهایت استفاده از دو روش مقدار دهی تصادفی و روش هی تفاوت چندانی در دقت نهایی داده های آموزش و تست برای این مساله ایجاد نکرد
## تاثیر تعداد داده های آموزشی
#### در این برنامه مساله با تقسیم داده ها به آموزشی و تست برای چند حالت مختلف انجام شد در صورت انتخاب تعداد داده های آموزشی به تعداد کم شبکه به طور مطلوبی آموزش نمیدید از طرفی انتخاب بیش از حد معمول تعداد داده های آموزشی موجب بر هم خوردن تعادل نسبت بین داده های آموزشی و تست می گردد و موجب می شود معیار درستی برای دقت عملکرد شبکه به دست نیاید در این مساله داده های آموزشی به تعداد 60000 در نهایت نتایج مطلوبی به دست داد هرجند انتخاب داده ها به میزان کمتر و در حدود بین 40000 تا 60000 تفاوت چندانی در نتایج ایجاد نکرد    

    
# نتایج بهترین وزنها و بایاسهای به دست آمده ذخیره شده و به پیوست ارسال شده است 
    
    
    
    